In [1]:
import decimal
import pandas as pd

## Transformation was Done by Alteryx. Results were Saved as CSVs.

In [92]:
# Load result CSVs into DataFrames
df_Facts_EV = pd.read_csv('Facts_EV.csv')
df_Facts_EV.columns = df_Facts_EV.columns.str.replace(' ', '_')
df_Facts_EV["Electric_Range"] = df_Facts_EV["Electric_Range"].apply(context.create_decimal_from_float)
df_Facts_EV["Base_MSRP"] = df_Facts_EV["Base_MSRP"].apply(context.create_decimal_from_float)
print(df_Facts_EV.head())
df_Facts_EV.dtypes

    Fact_ID Electric_Range Base_MSRP  Date_ID Location_ID   Vehicle_ID  \
0  F0121432             76         0  D000011     L000445  V0140745573   
1  F0115354            204         0  D000017     L000505  V0256365756   
2  F0132177            220         0  D000017     L000354  V0329296670   
3  F0105138            239         0  D000017     L000308  V0107280638   
4  F0144595            200         0  D000014     L000445  V0145529142   

  EV_Type_ID  
0    E000029  
1    E000029  
2    E000029  
3    E000029  
4    E000029  


Fact_ID           object
Electric_Range    object
Base_MSRP         object
Date_ID           object
Location_ID       object
Vehicle_ID        object
EV_Type_ID        object
dtype: object

In [64]:
df_Vehicle_Dim = pd.read_csv('Vehicle_Dim.csv')
df_Vehicle_Dim.columns = df_Vehicle_Dim.columns.str.replace(' ', '_')
print(df_Vehicle_Dim.head())
df_Vehicle_Dim.dtypes

    Vehicle_ID         VIN   Make     Model  DOL_Vehicle_ID
0  V0235085336  WAUTPBFF4H   AUDI        A3       235085336
1  V0237896795  WAUUPBFF2J   AUDI        A3       237896795
2  V0154498865  5YJSA1E22H  TESLA   MODEL S       154498865
3  V0154525493  1C4JJXP62M   JEEP  WRANGLER       154525493
4  V0225996361  5YJ3E1EC9L  TESLA   MODEL 3       225996361


Vehicle_ID        object
VIN               object
Make              object
Model             object
DOL_Vehicle_ID     int64
dtype: object

In [86]:
df_Location_Dim = pd.read_csv('Location_Dim.csv')
df_Location_Dim.columns = df_Location_Dim.columns.str.replace(' ', '_')
context = decimal.Context(prec=7)
df_Location_Dim["Longitude"] = df_Location_Dim["Longitude"].apply(context.create_decimal_from_float)
df_Location_Dim["Latitude"] = df_Location_Dim["Latitude"].apply(context.create_decimal_from_float)
print(df_Location_Dim.head())
df_Location_Dim.dtypes

  Location_ID      County              City State  Postal_Code  Longitude  \
0     L000001  Las Animas          Trinidad    CO        81082  -104.5165   
1     L000002     El Paso  Colorado Springs    CO        80925  -104.6434   
2     L000003     El Paso  Colorado Springs    CO        80908  -104.7143   
3     L000004     El Paso  Colorado Springs    CO        80923  -104.7172   
4     L000005       Adams            Aurora    CO        80019  -104.7338   

   Latitude  
0  37.16826  
1  38.74197  
2  39.03137  
3  38.92589  
4  39.77314  


Location_ID    object
County         object
City           object
State          object
Postal_Code     int64
Longitude      object
Latitude       object
dtype: object

In [61]:
df_EV_Type_Dim = pd.read_csv('Electric_Vehicle_Type_Dim.csv')
df_EV_Type_Dim.rename(columns={'Clean Alternative Fuel Vehicle (CAFV) Eligibility': 'CAFV Eligibility'}, inplace=True)
df_EV_Type_Dim.columns = df_EV_Type_Dim.columns.str.replace(' ', '_')
print(df_EV_Type_Dim.head())
df_EV_Type_Dim.dtypes

  EV_Type_ID           Electric_Vehicle_Type  \
0    E000001  Battery Electric Vehicle (BEV)   
1    E000002  Battery Electric Vehicle (BEV)   
2    E000003  Battery Electric Vehicle (BEV)   
3    E000004  Battery Electric Vehicle (BEV)   
4    E000005  Battery Electric Vehicle (BEV)   

                          CAFV_Eligibility CAFV_Eligibility_Type  \
0  Clean Alternative Fuel Vehicle Eligible              Eligible   
1  Clean Alternative Fuel Vehicle Eligible              Eligible   
2  Clean Alternative Fuel Vehicle Eligible              Eligible   
3  Clean Alternative Fuel Vehicle Eligible              Eligible   
4  Clean Alternative Fuel Vehicle Eligible              Eligible   

                                    Electric_Utility  
0                                        AVISTA CORP  
1  BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...  
2  BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...  
3  BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...  
4  BONNEVILLE POWER ADMINIS

EV_Type_ID               object
Electric_Vehicle_Type    object
CAFV_Eligibility         object
CAFV_Eligibility_Type    object
Electric_Utility         object
dtype: object

In [59]:
df_Date_Dim = pd.read_csv('Date_Dim.csv')
df_Date_Dim.columns = df_Date_Dim.columns.str.replace(' ', '_')
print(df_Date_Dim.head())
df_Date_Dim.dtypes

   Date_ID  Model_Year
0  D000001        1997
1  D000002        1998
2  D000003        1999
3  D000004        2000
4  D000005        2002


Date_ID       object
Model_Year     int64
dtype: object

## Loading Fact Table and Dim Tables into my GCP DW

In [56]:
from google.cloud import storage
from datetime import datetime
from google.cloud import bigquery
from io import StringIO
from google.oauth2 import service_account
import pandas_gbq
import json
from pandas_gbq import to_gbq
from google.cloud import bigquery
import os

In [52]:
def upload_table_to_bq(df, project_id, dataset_name, table_name):
    client = bigquery.Client(project=project_id)
    dataset_id = f'{project_id}.{dataset_name}'
    table_id = f'{dataset_id}.{table_name}'

    try:
        # Ensure the dataset exists
        dataset = bigquery.Dataset(dataset_id)
        client.create_dataset(dataset, exists_ok=True)

        # Load the DataFrame to BigQuery
        job = client.load_table_from_dataframe(df, table_id)
        job.result()  # Wait for the job to complete
        print("Table uploaded to BigQuery.")
    except Exception as e:
        print(f'Failed to upload table: {e}')

In [66]:
# Set variables
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "prime-task-421716-f42265232751.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

project_id = 'prime-task-421716'

prime-task-421716-f42265232751.json


In [60]:
# Loading
upload_table_to_bq(df_Date_Dim, project_id, 'ev_population_data', 'Date_Dim')

prime-task-421716-f42265232751.json
Table uploaded to BigQuery.


In [70]:
upload_table_to_bq(df_Vehicle_Dim, project_id, 'ev_population_data', 'Vehicle_Dim')

Table uploaded to BigQuery.


In [88]:
upload_table_to_bq(df_Location_Dim, project_id, 'ev_population_data', 'Location_Dim')

Table uploaded to BigQuery.


In [90]:
upload_table_to_bq(df_EV_Type_Dim, project_id, 'ev_population_data', 'Electric_Vehicle_Type_Dim')

Table uploaded to BigQuery.


In [93]:
upload_table_to_bq(df_Facts_EV, project_id, 'ev_population_data', 'Facts_EV')

Table uploaded to BigQuery.
